In [ ]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

# Utility Functions

In [ ]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
    #print("Layer: {}".format(layer_obj.name))
    #print("Layer Start Ind: {}".format(layer_obj.istart))
    #print("Layer End Ind: {}".format(layer_obj.iend))
    #print(z_vals)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=sim_proc.X, axis=0)
    y_integral = intg.trapz(x_integral, x=sim_proc.Y, axis=0)
    return y_integral

def integrate_endpoints(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    Nx = arr_slice.shape[1]
    Ny = arr_slice.shape[2]
    Nz = arr_slice.shape[0]
    extended = np.zeros((Nz, Nx+1, Ny+1), dtype=arr_slice.dtype)
    extended[:, 0:Nx, 0:Ny] = arr_slice
    extended[:, 0:Nx, -1] = arr_slice[:, 0:Nx, 0]
    extended[:, -1, 0:Ny] = arr_slice[:, 0, 0:Ny]
    extended[:, -1, -1] = arr_slice[:, 0, 0]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = sim_proc.Z[layer_obj.istart:layer_obj.iend]
    x_vals = np.append(sim_proc.X, sim_proc.X[-1]+sim_proc.dx)
    y_vals = np.append(sim_proc.Y, sim_proc.Y[-1]+sim_proc.dy)
    z_integral = intg.trapz(extended, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

def compute_fluxes(sim):
    fluxes = sim.data['fluxes']
    total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
    total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
    total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
    total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
    print('Total Incident Power = {}'.format(total_incident_power))
    print('Total Reflected Power = {}'.format(total_reflected_power))
    print('Total Transmitted Power = {}'.format(total_transmitted_power))
    print('Total Absorbed Power = {}'.format(total_absorbed_power))
    summed_absorbed_power = 0
    abs_dict_fluxmethod = {}
    for layer, (forw_top, back_top) in fluxes.items():
        if '_bottom' in layer:
            continue
        bottom = layer+'_bottom'
        forw_bot, back_bot = fluxes[bottom] 
        print('-'*25)
        print('Layer: {}'.format(layer))
        print('Forward Top: {}'.format(forw_top))
        print('Backward Top: {}'.format(back_top))
        print('Forward Bottom: {}'.format(forw_bot))
        print('Backward Bottom: {}'.format(back_bot))
        P_in = forw_top + -1*back_bot
        P_out = forw_bot + -1*back_top
        print('Power Entering Layer: {}'.format(P_in))
        print('Power Leaving Layer: {}'.format(P_out))
        P_lost = P_in - P_out
        P_abs = .5*P_lost*(sim.period**2)/Zo
        abs_dict_fluxmethod[layer] = P_abs 
        print('Absorbed in Layer: {}'.format(P_abs))
        summed_absorbed_power += P_abs
    print('-'*25)
    print('Summed Absorption= {}'.format(summed_absorbed_power))
    return abs_dict_fluxmethod, summed_absorbed_power

# NW w/ Shell

## No Endpoints

In [ ]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [ ]:
sim.get_fluxes()

In [ ]:
sim.data['fluxes'] = {record[0].decode():(record[1], record[2]) for record in sim.data['fluxes']}

In [ ]:
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
normSlice = sim_proc.data['normEsquared'][:, :, 62]
plt.matshow(normSlice)
plt.show()

In [ ]:
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    if np.isnan(res):
        print("Result is nan!")
        res = 0
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

## Include Endpoints

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

# NW w/ Shell w/ Lanczos

In [ ]:
conf = Config('AbsorptionTest.yml')
conf[('Solver', 'LanczosSmoothing')] = True
sim = Simulator(conf)
sim.setup()
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

## No Endpoints

In [ ]:
sim.get_fluxes()

In [ ]:
abs_dict_fluxmethod, flux_method_total = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

## Include Endpoints

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
int_method_total = 0
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate_endpoints(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)
    int_method_total += p_abs_imag
print("Integral Method Total Absorption: {}".format(int_method_total))

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))
pdiff_total = 100*abs(flux_method_total - int_method_total)/flux_method_total
print('-'*25)
print("Total Percent Difference: {}".format(pdiff_total))

# NW no Shell

In [ ]:
conf = Config('AbsorptionTestnoshell.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [ ]:
sim.get_fluxes()

In [ ]:
abs_dict_fluxmethod = compute_fluxes(sim)

In [ ]:
sim.get_field()

In [ ]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))

In [ ]:
print(sim.data['fluxes'])

In [ ]:
print(sim.flux_dict)

In [ ]:
simulation = Simulation(simulator=sim)
print(simulation.data['fluxes'])
print(simulation.layers)

In [ ]:
alist = [1., 2., 3.]
print('{}, {}, {}, {}'.format('test', *alist))